In [2]:
!pip install  langchain langchain_community huggingface huggingface_hub pypdf rank_bm25

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.2/313.2 kB 29.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.0 MB/s eta 0:00:00


In [3]:
!pip install langchain_huggingface

In [4]:
from langchain.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.retrievers import BM25Retriever
from langchain_huggingface import HuggingFaceEmbeddings ,HuggingFacePipeline, ChatHuggingFace
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA


In [5]:
from huggingface_hub import login

from google.colab import userdata
login(token=userdata.get("HUGGINGFACEHUB_API_TOKEN"))

# Load LLM and Embedding model locally:✅

In [6]:

llm = HuggingFacePipeline.from_model_id(
    model_id = 'TinyLlama/TinyLlama-1.1B-Chat-v1.0',
        task='text-generation',
        pipeline_kwargs={'temperature':0.1}

)

model = ChatHuggingFace(llm=llm)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Device set to use cuda:0


In [7]:
embedding_model = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [8]:
loader = PyPDFLoader('/content/Cross Validation Techniques - Ataullha.pdf')
document = loader.load()

In [9]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
chunks = text_splitter.split_documents(document)

for chunk in chunks:
    chunk.page_content = chunk.page_content.replace('\n', '')

# Create ketword/laxical retrival using BM25 (updated version of IF-IDF)

In [10]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [11]:
from nltk import word_tokenize

In [12]:
bm25_retriever = BM25Retriever.from_documents(chunks ,**{'k':2},preprocess_func=word_tokenize)

In [13]:
bm25_retriever.invoke('Reducing Data Leakage:')[0].page_content

'4.  Utilizing  Limited  Data  Effectively:  In  scenarios  where  the  dataset  is  small,  cross-validation  is beneficial. By partitioning the data into multiple folds, we can effectively use the available data for both training and testing purposes without compromising the model evaluation. 5.  Reducing  Data  Leakage:  Data  leakage  refers  to  situations  where  information  from  the  test  set inadvertently influences the'

In [14]:
bm25_retriever.invoke('Reducing Data Leakage:')[1].page_content

"but provides an unbiased estimate of a model's performance and is particularly useful for small datasets.  Comparison  of  Cross-Validation  Techniques Cross-Validation  Technique  Computational  Cost  Data  Utilization  Hold-Out  Cross-Validation  Low  to  Moderate  Suboptimal  K-Fold  Cross-Validation  Moderate  to  High  Good  Stratified  K-Fold  Cross-Validation  Moderate  to  High  Good"

# Create vactor store retriver


In [15]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 14.7 MB/s eta 0:00:00


In [16]:
vectorstore = FAISS.from_documents(documents=chunks, embedding=embedding_model)
vectore_retriver = vectorstore.as_retriever()

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [17]:
retriever_doc=vectore_retriver.invoke('Reducing Data Leakage')

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [18]:
retriever_doc[0].page_content

'4.  Utilizing  Limited  Data  Effectively:  In  scenarios  where  the  dataset  is  small,  cross-validation  is beneficial. By partitioning the data into multiple folds, we can effectively use the available data for both training and testing purposes without compromising the model evaluation. 5.  Reducing  Data  Leakage:  Data  leakage  refers  to  situations  where  information  from  the  test  set inadvertently influences the'

# Hybrid search


In [19]:
from langchain.retrievers import EnsembleRetriever


ensemble_retriever = EnsembleRetriever(
    retrievers=[bm25_retriever, vectore_retriver],
    weights=[0.5, 0.5]
)

In [20]:
chain = RetrievalQA.from_chain_type(llm=model, chain_type='stuff', retriever=ensemble_retriever)

In [21]:
chain.invoke("what is Reducing Data Leakage")

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


{'query': 'what is Reducing Data Leakage',
 'result': '<|system|>\nUse the following pieces of context to answer the user\'s question.\nIf you don\'t know the answer, just say that you don\'t know, don\'t try to make up an answer.\n----------------\n4.  Utilizing  Limited  Data  Effectively:  In  scenarios  where  the  dataset  is  small,  cross-validation  is beneficial. By partitioning the data into multiple folds, we can effectively use the available data for both training and testing purposes without compromising the model evaluation. 5.  Reducing  Data  Leakage:  Data  leakage  refers  to  situations  where  information  from  the  test  set inadvertently influences the\n\nbut provides an unbiased estimate of a model\'s performance and is particularly useful for small datasets.  Comparison  of  Cross-Validation  Techniques Cross-Validation  Technique  Computational  Cost  Data  Utilization  Hold-Out  Cross-Validation  Low  to  Moderate  Suboptimal  K-Fold  Cross-Validation  Modera

# compression
Contextual compression is self-explanatory, which means that it compresses the relevant (context) data from the large, raw dataset.

Filtering is a process that further cleans the data after compression from the large dataset. Both in sync can help us drastically reduce storage, retrieval, and cost of operation.

# compression pipeline

In [22]:
from langchain.retrievers.document_compressors import DocumentCompressorPipeline
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import EmbeddingsFilter
from langchain.document_transformers import EmbeddingsRedundantFilter

In [23]:
redundant_filter = EmbeddingsRedundantFilter(
    embeddings=embedding_model,
    similarity_threshold=0.75
)
relevency_filter = EmbeddingsFilter(
    embeddings=embedding_model,
    k=5
)


compression_pipeline = DocumentCompressorPipeline(
     transformers=[redundant_filter,relevency_filter]
 )

In [24]:
compression_pip_retriver = ContextualCompressionRetriever(
    base_compressor = compression_pipeline,
    base_retriever= ensemble_retriever
)

In [26]:
com_context=compression_pip_retriver.invoke('Reducing Data Leakage')

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [33]:
for doc in com_context:
  print(doc.page_content)
  print('\n')

4.  Utilizing  Limited  Data  Effectively:  In  scenarios  where  the  dataset  is  small,  cross-validation  is beneficial. By partitioning the data into multiple folds, we can effectively use the available data for both training and testing purposes without compromising the model evaluation. 5.  Reducing  Data  Leakage:  Data  leakage  refers  to  situations  where  information  from  the  test  set inadvertently influences the


training process. Cross-validation helps minimize data leakage because each data point is used in both training and testing at different times, reducing the chances of the model "cheating" by memorizing specific examples. 6.  Model  Selection:  Cross-validation  is  valuable  when  comparing  multiple  models  to  determine  which one performs the best on unseen data. Instead of relying solely on the


Stratified  K-Fold  Cross-Validation  addresses  this  issue  by  preserving  the  class  distribution  within  each fold. The basic idea is to maintain the c